In [2]:


# Load engagement data from Task 2
engagement_data = pd.read_excel('../data/telecom_data.xlsx')

# Load experience data from Task 3 (as we are assuming no clusters exist yet)
experience_data = pd.read_excel('../data/telecom_data.xlsx')

# Inspect the data
print(engagement_data.head())
print(experience_data.head())

              Bearer Id               Start  Start ms                 End  \
0  13114483460844900352 2019-04-04 12:01:18     770.0 2019-04-25 14:35:31   
1  13114483482878900224 2019-04-09 13:04:04     235.0 2019-04-25 08:15:48   
2  13114483484080500736 2019-04-09 17:42:11       1.0 2019-04-25 11:58:13   
3  13114483485442799616 2019-04-10 00:31:25     486.0 2019-04-25 07:36:35   
4  13114483499480700928 2019-04-12 20:10:23     565.0 2019-04-25 10:40:32   

   End ms  Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0   662.0  1823652.0  2.082014e+14   3.366496e+10  3.552121e+13   
1   606.0  1365104.0  2.082019e+14   3.368185e+10  3.579401e+13   
2   652.0  1361762.0  2.082003e+14   3.376063e+10  3.528151e+13   
3   171.0  1321509.0  2.082014e+14   3.375034e+10  3.535661e+13   
4   954.0  1089009.0  2.082014e+14   3.369980e+10  3.540701e+13   

  Last Location Name  ...  Youtube DL (Bytes)  Youtube UL (Bytes)  \
0   9164566995485190  ...          15854611.0           2501332.0

In [4]:
import sys
import os
sys.path.append(os.path.abspath('../scripts'))
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import numpy as np

In [5]:
# Load the dataset
from load_data import load_data_from_postgres

In [6]:
# Define your SQL query
query = "SELECT * FROM xdr_data;" # Replace with your actual table name.

# Load data from PostgreSQL
df = load_data_from_postgres(query)

# Display the first few rows of the dataframe
if df is not None:
    print("Successfully loaded the data.")
else:
    print("Failed to load data.")

c:\Users\biement fanteye\Desktop\TelloCello\scripts\load_data.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Successfully loaded the data.


In [8]:
# Define your SQL queries for engagement and experience data
query_engagement = "SELECT * FROM xdr_data;"  # Replace with the actual table name for engagement data
query_experience = "SELECT * FROM xdr_data;"  # Replace with the actual table name for experience data

# Load data from PostgreSQL
engagement_data = load_data_from_postgres(query_engagement)
experience_data = load_data_from_postgres(query_experience)

# Inspect the data
if engagement_data is not None:
    print("Successfully loaded engagement data.")
else:
    print("Failed to load engagement data.")

if experience_data is not None:
    print("Successfully loaded experience data.")
else:
    print("Failed to load experience data.")


c:\Users\biement fanteye\Desktop\TelloCello\scripts\load_data.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Successfully loaded engagement data.
Successfully loaded experience data.


In [9]:
# Task 4.1: Use alternative engagement features
engagement_features = ['Total UL (Bytes)', 'Total DL (Bytes)', 'Activity Duration DL (ms)', 'Activity Duration UL (ms)']

# Fill NaN values with the mean of each column
engagement_data[engagement_features] = engagement_data[engagement_features].fillna(engagement_data[engagement_features].mean())

# Standardize the engagement features
scaler = StandardScaler()
engagement_scaled = scaler.fit_transform(engagement_data[engagement_features])

# Perform KMeans clustering (initial clustering for engagement)
kmeans_engagement = KMeans(n_clusters=3, random_state=42)
engagement_data['engagement_cluster'] = kmeans_engagement.fit_predict(engagement_scaled)

# Get cluster centers and calculate distance to least engaged cluster
from sklearn.metrics.pairwise import euclidean_distances
least_engaged_cluster = np.argmin(kmeans_engagement.cluster_centers_.sum(axis=1))
engagement_data['engagement_score'] = euclidean_distances(engagement_scaled, [kmeans_engagement.cluster_centers_[least_engaged_cluster]]).flatten()

# Inspect engagement data
print(engagement_data[['MSISDN/Number', 'engagement_cluster', 'engagement_score']].head())

   MSISDN/Number  engagement_cluster  engagement_score
0   3.366496e+10                   1          0.521900
1   3.368185e+10                   0          2.040323
2   3.376063e+10                   1          1.204255
3   3.375034e+10                   0          2.492466
4   3.369980e+10                   0          1.378744


In [10]:
experience_data.columns

Index(['Bearer Id', 'Start', 'Start ms', 'End', 'End ms', 'Dur. (ms)', 'IMSI',
       'MSISDN/Number', 'IMEI', 'Last Location Name', 'Avg RTT DL (ms)',
       'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)',
       'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)',
       'DL TP < 50 Kbps (%)', '50 Kbps < DL TP < 250 Kbps (%)',
       '250 Kbps < DL TP < 1 Mbps (%)', 'DL TP > 1 Mbps (%)',
       'UL TP < 10 Kbps (%)', '10 Kbps < UL TP < 50 Kbps (%)',
       '50 Kbps < UL TP < 300 Kbps (%)', 'UL TP > 300 Kbps (%)',
       'HTTP DL (Bytes)', 'HTTP UL (Bytes)', 'Activity Duration DL (ms)',
       'Activity Duration UL (ms)', 'Dur. (ms).1', 'Handset Manufacturer',
       'Handset Type', 'Nb of sec with 125000B < Vol DL',
       'Nb of sec with 1250B < Vol UL < 6250B',
       'Nb of sec with 31250B < Vol DL < 125000B',
       'Nb of sec with 37500B < Vol UL',
       'Nb of sec with 6250B < Vol DL < 31250B',
       'Nb of sec with 6250B < Vol UL < 37500B',


In [11]:
# Define experience features
experience_features = ['Avg RTT DL (ms)', 'Avg Bearer TP DL (kbps)', 'TCP DL Retrans. Vol (Bytes)']

# Fill missing values with the mean of each column
experience_data[experience_features] = experience_data[experience_features].fillna(experience_data[experience_features].mean())


# Normalize the data
scaler = StandardScaler()
experience_scaled = scaler.fit_transform(experience_data[experience_features])

# Perform KMeans clustering for experience
kmeans_experience = KMeans(n_clusters=3, random_state=42)
experience_data['experience_cluster'] = kmeans_experience.fit_predict(experience_scaled)

# Get cluster centers and calculate distance to the worst experience cluster
worst_experience_cluster = np.argmin(kmeans_experience.cluster_centers_.sum(axis=1))
experience_data['experience_score'] = np.linalg.norm(experience_scaled - kmeans_experience.cluster_centers_[worst_experience_cluster], axis=1)

# Display the first few rows of the resulting DataFrame
print("\nExperience Data with Clusters and Scores:")
print(experience_data[['MSISDN/Number', 'experience_cluster', 'experience_score']].head())



Experience Data with Clusters and Scores:
   MSISDN/Number  experience_cluster  experience_score
0   3.366496e+10                   0          0.561880
1   3.368185e+10                   0          0.555890
2   3.376063e+10                   0          0.552732
3   3.375034e+10                   0          0.551151
4   3.369980e+10                   0          0.552732


In [12]:
# Merge engagement and experience data on user ID
merged_data = pd.merge(engagement_data[['MSISDN/Number', 'engagement_score']], 
                       experience_data[['MSISDN/Number', 'experience_score']], 
                       on='MSISDN/Number', how='inner')  # Use 'inner' join to keep only matching records

# Check for any remaining missing values in the merged data
print("Missing values in merged data:")
print(merged_data.isnull().sum())

# Remove rows with missing values
merged_data = merged_data.dropna()

# Calculate satisfaction score as average of engagement and experience score
merged_data['satisfaction_score'] = (merged_data['engagement_score'] + merged_data['experience_score']) / 2

# Find top 10 most satisfied customers
top_10_satisfied = merged_data.nlargest(10, 'satisfaction_score')
print(top_10_satisfied)

Missing values in merged data:
MSISDN/Number       1136356
engagement_score          0
experience_score          0
dtype: int64
        MSISDN/Number  engagement_score  experience_score  satisfaction_score
484883   3.366232e+10          0.644179        173.100680           86.872430
484480   3.366087e+10          2.061627        115.384950           58.723288
691514   3.366087e+10          0.718562        115.384950           58.051756
50500    3.368369e+10          1.669842         97.877180           49.773511
751960   3.366773e+10         18.738119         31.741629           25.239874
14688    3.376264e+10         16.707330         32.175261           24.441295
388562   3.376094e+10          0.760631         48.040095           24.400363
507915   3.366045e+10         11.744362         37.025443           24.384902
8831     3.366045e+10         11.735380         37.025443           24.380412
405353   3.360679e+10          0.456561         46.752673           23.604617


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Use engagement and experience scores to predict satisfaction score
X = merged_data[['engagement_score', 'experience_score']]
y = merged_data['satisfaction_score']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and fit the regression model
reg_model = LinearRegression()
reg_model.fit(X_train, y_train)

# Predict satisfaction score
y_pred = reg_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 3.472490325829538e-29


In [14]:
# Perform KMeans clustering on engagement and experience scores
kmeans_scores = KMeans(n_clusters=2, random_state=42)
merged_data['score_cluster'] = kmeans_scores.fit_predict(merged_data[['engagement_score', 'experience_score']])

# Inspect the clusters
print(merged_data.groupby('score_cluster').mean())

               MSISDN/Number  engagement_score  experience_score  \
score_cluster                                                      
0               3.810032e+10          1.827094          0.833340   
1               3.368623e+10          4.413590         22.129761   

               satisfaction_score  
score_cluster                      
0                        1.330217  
1                       13.271675  


In [22]:
# Aggregate the average satisfaction and experience score per cluster
cluster_aggregates = merged_data.groupby('score_cluster').agg(
    avg_satisfaction=('satisfaction_score', 'mean'),
    avg_experience=('experience_score', 'mean')
)
print(cluster_aggregates)

               avg_satisfaction  avg_experience
score_cluster                                  
0                      1.289463        0.954016
1                     50.634934       99.697103
